In [16]:
"""
3 / 5 (순서 / 메모리 종류)
ConversationSummaryMemory
llm을 사용하기에 비용이 든다.
message를 그대로 저장하는 것이 아니라, 요약해서 저장한다.
장점: 많은 양의 대화에서 토큰 사용이 현저히 줄어든다.(요약)
단점: 적은 양의 대화에서 토큰 사용이 상대적으로 많다.

"""

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
  memory.save_context({"input": input}, {"output": output})

def get_history():
  return memory.load_memory_variables({})

add_message("Hi I'm bigPerson, I live in South Korea", "Wow that is so cool!")


In [17]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [18]:
get_history()

{'history': 'The human introduces themselves as bigPerson from South Korea. The AI responds with enthusiasm, finding it cool and expressing a desire to visit the country because it is so pretty.'}